In [28]:
import os
from pathlib import Path
import geopandas
import subprocess
import logging
import requests
import gzip
import fiona
from typing import Union
from ipyleaflet import Map, GeoData, basemaps, LayersControl


In [8]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR = Path(os.getcwd())
BASE_DIR = f"{WORK_DIR.parents[3]}/work/datasets"
logging.basicConfig(level=logging.INFO)

# @TODO: Add expected data files source as an environment variable.
assert (
    BASE_DIR == "/home/jovyan/work/datasets"
), f"{BASE_DIR} is not the correct directory"

input_dir = Path(f"{BASE_DIR}/raw/extent-layer-creation")
output_dir = Path(f"{BASE_DIR}/processed/extent-layer-creation")

input_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

## Path setup
url = "https://www.dropbox.com/s/g8n05ll44cy2od1/gmw_v3_fnl_mjr_v314.gpkg.gz?dl=0"
d_file = Path(f"{input_dir}/gmw_v3_fnl_mjr_v314.gpkg.gz")
in_file = d_file.with_suffix("")
out_file = Path(f"{output_dir}/{in_file.name}").with_suffix(".mbtiles")


In [13]:
def downloadData(
    file_path: str, download_url: str, update: bool = False
) -> Union[int, str]:
    """
    Download a file to a path.
    Parameters
    ----------
    file_path : str - The path to the file to download.
    update : bool, optional - If True, the file will be downloaded again even if it already exists.
                            The default is False.

    Returns
    -------
    int - 0 if the file was downloaded successfully, 1 if the file download failed.
    """
    try:
        if update or not os.path.exists(file_path):
            logging.info("Downloading data...")

            response = requests.get(download_url, stream=True)

            with open(file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=128):
                    f.write(chunk)
        else:
            logging.info("data already downloaded.")

        return 0
    except Exception as e:
        logging.error(e)
        return 1


In [5]:
def mbtilesGeneration(
    data_path: Path, output_path: Union[Path, None] = None, update: bool = False
) -> Path:
    """
    Simplify geometry of a GeoDataFrame using tippecanoe.
    Parameters
    ----------
    data_path : Path - The path to the GeoDataFrame to simplify.
    output_path : Path - The path to the output GeoDataFrame.
    update : bool, optional - If True, the output GeoDataFrame will be overwritten.
                            The default is False.

    Returns
    -------
    Path - The path to the generated mbtiles file.
    """
    try:
        assert data_path.exists(), "Data path does not exist."

        if not output_path:
            output_path = data_path.with_suffix(".mbtiles")

        if update or not output_path.exists():

            if data_path.suffix != ".json":
                CMD = f'mapshaper {data_path} -clean allow-overlaps rewind -o format=geojson {data_path.with_suffix(".json")} force'
                subprocess.run(CMD, shell=True, check=True)
                data_path = data_path.with_suffix(".json")

            assert data_path.suffix == ".json", "Data path must be a json file."

            logging.info("Creating mbtiles file...")

            subprocess.run(
                f"tippecanoe -zg -f -P -o {output_path} --extend-zooms-if-still-dropping {data_path}",
                shell=True,
                check=True,
            )
            data_path.unlink(exist_ok=True)

        return output_path

    except Exception as e:
        logging.error(e)
        return 1


Tiles for extent layer creation pipe.

Download the vector data


In [53]:
r = requests.post(
    "https://www.dropbox.com/sharing/fetch_user_content_link",
    data={"is_xhr": True, "t": "J2MB3dUwoB7nchfSip2OOjnR", "url": url, "origin": ""},
    headers={"content-type": "application/x-www-form-urlencoded; charset=UTF-8"},
    cookies={
        "gvc": "J2MB3dUwoB7nchfSip2OOjnR",
        "t": "J2MB3dUwoB7nchfSip2OOjnR",
        "__Host-js_csrf": "J2MB3dUwoB7nchfSip2OOjnR",
        "__Host-ss": "60irgIKG-k",
        "locale": "en",
        "__Host-logged-out-session": "ChD5IC9J1UubKzaH1kNpzIlyEOvw%2F5YGGi5BRC13a2RIUlJxeV80SHVOWFpEcWJoQXhfQ1J3NDMybGh1bTRrWnRSTE5ZWHhB",
    },
)
download_url = r.text

downloadData(d_file, download_url, True)


INFO:root:Downloading data...


0

Converting to json


In [57]:
with gzip.open(d_file) as f:
    features_train = geopandas.read_file(f, driver="GPKG")


In [61]:
features_train.to_file(in_file.with_suffix(".json"), driver="GeoJSON")


MBtile generation

In [66]:
mbtilesGeneration(in_file.with_suffix(".json"), out_file, update=True)


INFO:root:Creating mbtiles file...
For layer 0, using name "gmw_v3_fnl_mjr_v314"
/home/jovyan/work/datasets/raw/extent-layer-creation/gmw_v3_fnl_mjr_v314.json:74605: Found ] at top level
/home/jovyan/work/datasets/raw/extent-layer-creation/gmw_v3_fnl_mjr_v314.json:63649: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
1121419 features, 241475136 bytes of geometry, 3364273 bytes of separate metadata, 176 bytes of string pool
Choosing a maxzoom of -z8 for features about 1100 feet (336 meters) apart
Choosing a maxzoom of -z11 for resolution of about 150 feet (45 meters) within features
  99.9%  11/161/1120   


PosixPath('/home/jovyan/work/datasets/processed/extent-layer-creation/gmw_v3_fnl_mjr_v314.mbtiles')

In [ ]:
#!npm install -g @mapbox/mbview

#!mbview $outputPath_mbtiles
